# Customer Support Agent implmented utilising webscraper tool from crewAI



In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from dotenv import load_dotenv
from utils import get_openai_api_key
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-mini'
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


## Creating Agents: Support Representative and Support Quality Assurance

In [7]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Xoxoday (https://www.xoxoday.com/) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [8]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Xoxoday (https://www.xoxoday.com/) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

Importing tools from ```crewai_tools```

In [9]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

Scraping support site with url

In [10]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.xoxoday.com/support"
)

# Assign Task to respective agents

In [11]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


Create and validating the crew


In [14]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=False,
  memory=True
)

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


Run the crew with sample input

In [15]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with redeeming a xoxocode "
               "I received for a course I purchased. "
                "I am not sure how to use it. "
                "I would like to know the steps to redeem it. "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with redeeming a xoxocode I received for a course I purchased. I am not sure how to use it. I would like to know the steps to redeem it. Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Thought: To provide accurate and complete guidance on redeeming a XoxoCode, I should look at the official support documentation from Xoxoday. This will ensure that the steps I provide are up-to-date and reliable. I'll check the support section on the Xoxoday website.
## Using tool: Read website content
## Tool Input: 
"{\"url\": \"https://www.xoxoday.com/support\"}"
## Tool Output: 
Support
Products
Loyalife Customer Loyalty Platform
Plum 

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
Hello Andrew,

Thank you for reaching out to us with your question about redeeming your XoxoCode for the course you purchased through DeepLearningAI. I’m happy to guide you through the process so you can access your course smoothly and without any hassle.

Here’s exactly how you can redeem your XoxoCode:

1. Go to the Xoxoday reward redemption storefront at [stores.xoxoday.com](https://stores.xoxoday.co

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Display Output

In [ ]:
from IPython.display import Markdown
result=str(result)
Markdown(result)

Hello Andrew,

Thank you for reaching out to us with your question about redeeming your XoxoCode for the course you purchased through DeepLearningAI. I’m happy to guide you through the process so you can access your course smoothly and without any hassle.

Here’s exactly how you can redeem your XoxoCode:

1. Go to the Xoxoday reward redemption storefront at [stores.xoxoday.com](https://stores.xoxoday.com).  
2. You do not need to create an account or log in; simply proceed using the guest checkout option.  
3. When prompted at checkout, enter your XoxoCode in the designated field to redeem your course reward.  
4. Follow the on-screen steps to complete the redemption process. Once successfully redeemed, you will gain access to your DeepLearningAI course.

The process is designed to be quick and user-friendly, so you can easily unlock the course associated with your code. If you run into any trouble during redemption—for example, if the code doesn’t work or you face technical issues—please don’t hesitate to reach out. I’m here to help and will gladly assist you until everything is working perfectly.

Also, feel free to keep this message handy for your reference, or visit our support page for more details on redeeming XoxoCodes at [Xoxoday Support - Plum Rewards](https://www.xoxoday.com/support).

Thanks again for choosing DeepLearningAI and Xoxoday! We’re excited to have you onboard and committed to making your experience smooth and enjoyable.

Warm regards,  
[Your Name]  
Senior Support Representative, Xoxoday

Reference: Information sourced from the official Xoxoday support website’s Plum rewards section on how to redeem reward codes ([https://www.xoxoday.com/support](https://www.xoxoday.com/support))